https://github.com/dnoak/contador-alevinos/blob/1733bc03d588966be7fa36fe853d7000c91447f3/src/models/detr/train_detr.py#L45

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

os.chdir('drive/MyDrive/9조 프로젝트/Data/Dataset')

In [ ]:
!pip install transformers
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 13.0 MB/s eta 0:00:00


In [ ]:
pip install timm==0.9.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.3 MB/s eta 0:00:00


In [ ]:
import timm
timm.__version__

'0.9.12'

In [ ]:
import torchvision
import os

class D_DETRDataset(torchvision.datasets.CocoDetection): # practice: img_folder = train
    def __init__(self, img_folder, processor, train = True):
        ann_file = os.path.join(img_folder, "labels.json")  # batch 때마다 directory에서 이미지 불러옴
        super(D_DETRDataset, self).__init__(img_folder, ann_file)
        self.processor = processor  # 이미지 변환

    def __len__(self):  # 데이터셋에 있는 이미지 총 수
        return len(self.ids)

    def __getitem__(self, idx):  # 특정 인덱스에 해당하는 이미지 및 타겟 반환
        img, target = super(D_DETRDataset, self).__getitem__(idx)

        # preprocess image and target (converting target to DETR format, resizing + normalization of both image and target)
        # preprocess에 입력되는 DetrImageProcessor의 img는 이미지 파일 자체, target은 COCO format 그대로 입력되어있어야 함.
        # fasterRCNN 구현할 때에는 한 이미지에 대한 여러 박스들을 학습 시에 하나한 매칭해주는 코드 구현해야 하는데 여기서는 자동으로 되나봄.
        image_id = self.ids[idx]
        target = {'image_id': image_id, 'annotations': target}
        encoding = self.processor(images=img, annotations=target, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze() # remove batch dimension
        target = encoding["labels"][0] # remove batch dimension

        return pixel_values, target

In [ ]:
from transformers import DeformableDetrImageProcessor

processor = DeformableDetrImageProcessor.from_pretrained("SenseTime/deformable-detr") # 이미지와 어노테이션을 처리하기 위한 프로세서 생성

train_dataset = D_DETRDataset(img_folder='train/data', processor=processor)
val_dataset = D_DETRDataset(img_folder='val/data', processor=processor, train=False)

preprocessor_config.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


loading annotations into memory...


In [ ]:
MODEL_PATH = '//content//drive//MyDrive//Personal Colab//민지'

In [ ]:
import pytorch_lightning as pl
from transformers import DeformableDetrForObjectDetection
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer

import torch

In [ ]:
def collate_fn(batch):
  pixel_values = [item[0] for item in batch]
  encoding = processor.pad(pixel_values, return_tensors="pt")
  labels = [item[1] for item in batch]
  batch = {}
  batch['pixel_values'] = encoding['pixel_values']
  batch['pixel_mask'] = encoding['pixel_mask']  # 어떤 부분이 padding인지를 나타내는 mask
  batch['labels'] = labels
  return batch

In [ ]:
class Model(pl.LightningModule):
    def __init__(self, lr, lr_backbone, weight_decay, id2label, train_dataloader, val_dataloader, CHECKPOINT = "facebook/detr-resnet-50"):
        super().__init__()
        if (CHECKPOINT == 'SenseTime/deformable-detr'):
            self.model = DeformableDetrForObjectDetection.from_pretrained(
                pretrained_model_name_or_path=CHECKPOINT,
                num_labels=len(id2label),
                ignore_mismatched_sizes=True
            )
        else:
            self.model = DetrForObjectDetection.from_pretrained(
            pretrained_model_name_or_path=CHECKPOINT,
            num_labels=len(id2label),
            ignore_mismatched_sizes=True
        )

        self.TRAIN_DATALOADER = train_dataloader
        self.VAL_DATALOADER = val_dataloader
        self.lr = lr
        self.lr_backbone = lr_backbone
        self.weight_decay = weight_decay

    def forward(self, pixel_values, pixel_mask):
        return self.model(pixel_values=pixel_values, pixel_mask=pixel_mask)

    def common_step(self, batch, batch_idx):
        pixel_values = batch["pixel_values"]
        pixel_mask = batch["pixel_mask"]
        labels = [{k: v.to(self.device) for k, v in t.items()} for t in batch["labels"]]

        outputs = self.model(pixel_values=pixel_values, pixel_mask=pixel_mask, labels=labels)

        loss = outputs.loss
        loss_dict = outputs.loss_dict

        return loss, loss_dict

    def training_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)
        # logs metrics for each training_step, and the average across the epoch
        self.log("training_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        for k,v in loss_dict.items():
            self.log("train_" + k, v.item())

        return loss

    def validation_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)
        self.log("validation/loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        for k, v in loss_dict.items():
            self.log("validation_" + k, v.item())

        return loss

    def configure_optimizers(self):
        # DETR authors decided to use different learning rate for backbone
        # you can learn more about it here:
        # - https://github.com/facebookresearch/detr/blob/3af9fa878e73b6894ce3596450a8d9b89d918ca9/main.py#L22-L23
        # - https://github.com/facebookresearch/detr/blob/3af9fa878e73b6894ce3596450a8d9b89d918ca9/main.py#L131-L139
        param_dicts = [
            {
                "params": [p for n, p in self.named_parameters() if "backbone" not in n and p.requires_grad]},
            {
                "params": [p for n, p in self.named_parameters() if "backbone" in n and p.requires_grad],
                "lr": self.lr_backbone,
            },
        ]
        return torch.optim.AdamW(param_dicts, lr=self.lr, weight_decay=self.weight_decay) # AdamW

    def train_dataloader(self):
        return self.TRAIN_DATALOADER

    def val_dataloader(self):
        return self.VAL_DATALOADER


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

class ModelTrainer():
    def __init__(self, log_every_n_steps, max_epochs, processor, model):
        TRAIN_DATASET = D_DETRDataset(img_folder='train/data', processor=processor, train=True)
        VAL_DATASET = D_DETRDataset(img_folder='val/data', processor=processor, train=False)

        TRAIN_DATALOADER = DataLoader(dataset=TRAIN_DATASET, collate_fn=collate_fn, batch_size=2, num_workers=4, shuffle=True, pin_memory=True)
        VAL_DATALOADER = DataLoader(dataset=VAL_DATASET, collate_fn=collate_fn, batch_size=2, num_workers=4)

        categories = TRAIN_DATASET.coco.cats
        id2label = {k: v['name'] for k,v in categories.items()}
        self.processor = processor
        self.model = Model(lr=1e-4, lr_backbone=1e-5, weight_decay=1e-4, id2label=id2label, train_dataloader=TRAIN_DATALOADER, val_dataloader=VAL_DATALOADER, CHECKPOINT=model)
        early_stop_callback = EarlyStopping(monitor="validation/loss", patience=5)
        checkpoint_callback = ModelCheckpoint(dirpath=MODEL_PATH,
                                              filename='best-deformable-detr-{epoch:02d}-{val_loss:.2f}',
                                              save_top_k=1,
                                              monitor="validation/loss",
                                              verbose=True)

        self.trainer = Trainer(
            devices=1,
            accelerator="gpu",
            max_epochs=max_epochs,
            gradient_clip_val=0.1,
            log_every_n_steps=log_every_n_steps)
            # callbacks=[early_stop_callback, checkpoint_callback])

    def train(self, ckpt_path=None):
        self.trainer.fit(self.model, ckpt_path=ckpt_path)
        #self.model.model.save_pretrained(model_path)

    def collate_fn(self, batch):
        pixel_values = [item[0] for item in batch]
        encoding = self.processor.pad(pixel_values, return_tensors="pt")
        labels = [item[1] for item in batch]
        return {
            'pixel_values': encoding['pixel_values'],
            'pixel_mask': encoding['pixel_mask'],
            'labels': labels
        }

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
MODEL_NAME = 'SenseTime/deformable-detr'
t = ModelTrainer(5, 10, processor, MODEL_NAME)
#t.train()

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


Some weights of the model checkpoint at SenseTime/deformable-detr were not used when initializing DeformableDetrForObjectDetection: ['model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DeformableDetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DeformableDetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DeformableDetrForObjectDetection wer

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
t.train(MODEL_PATH, ckpt_path = '/content/drive/MyDrive/9조 프로젝트/Data/Dataset/lightning_logs/version_38/checkpoints/epoch=5-step=76482.ckpt')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-32cd93e430e1>", line 1, in <cell line: 1>
    t.train(MODEL_PATH, ckpt_path = '')
TypeError: ModelTrainer.train() got multiple values for argument 'ckpt_path'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr

In [ ]:
import torch
device = torch.device('cuda')
print(device)
print(torch.cuda.is_available())

cuda
True


In [ ]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir logs/

In [ ]:
#trainer.fit(model, ckpt_path = "DETR_log/lightning_logs/version_1/checkpoints/epoch=2-step=9561.ckpt")

In [ ]:
torch.max_split_size_mb=0